###Required Libraries

In [1]:
%%bash
pip install -q transformers einops accelerate langchain bitsandbytes
pip install openai
pip install pyperclip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 133.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Load the Libraries & Tokenizer

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

import pandas as pd
import pyperclip

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

Build the Model Pipeline using Hugging Face Transformers Pipeline

In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:788 in pipeline       │
│                                                                                                  │
│   785 │   # Forced if framework already defined, inferred if it's None                           │
│   786 │   # Will load the correct model if possible                                              │
│   787 │   model_classes = {"tf": targeted_task["tf"], "pt": targeted_task["pt"]}                 │
│ ❱ 788 │   framework, model = infer_framework_load_model(                                         │
│   789 │   │   model,                                                                             │
│   790 │   │   model_classes=model_classes,                                                       │
│   791 │   │   config=config,                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:279 in                    │
│ infer_framework_load_model                                                                       │
│                                                                                                  │
│    276 │   │   │   │   continue                                                                  │
│    277 │   │                                                                                     │
│    278 │   │   if isinstance(model, str):                                                        │
│ ❱  279 │   │   │   raise ValueError(f"Could not load model {model} with any of the following cl  │
│    280 │                                                                                         │
│    281 │   framework = "tf" if "keras.engine.training.Model" in str(inspect.getmro(model.__clas  │
│    282 │   return framework, model                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Could not load model tiiuae/falcon-7b-instruct with any of the following classes: (<class 
'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 
'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>).

### To Automize chatGPT prompting run cell below
* If no OpenAI API key presen, then do it manually by running `comaprisonManual` function.

In [ ]:
import os
import openai

openai.api_key = "<YOUR-API-KEY>"

def chatWithGPT(prompt):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
  {"role": "user", "content": prompt}
  ]
  )
  return completion.choices[0].message.content

### Model Inference using the pipeline that was built above

#### Falcon-7B model output generating function

In [ ]:
def chatWithFalcon(prompt):
  sequences = pipeline(
      prompt,
      max_length=200,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  output = sequences[0]['generated_text']
  return output[output.find('\n')+1:]

In [ ]:
from time import perf_counter
from datetime import datetime

In [ ]:
df = pd.DataFrame(columns = ['timestamp','prompt','falcon','chatGPT','elapsed_time_GPU','elapsed_time_CPU'])
def comparisonManual(prompt,is_gpu=True):
  # pyperclip.copy(prompt)
  case_no = len(df)

  t_start = perf_counter()
  falcon_output = chatWithFalcon(prompt)
  elapsed_time = perf_counter() - t_start

  if is_gpu:
    df.loc[case_no] = [datetime.now(), prompt, falcon_output, input(), elapsed_time, '']
  else:
    df.loc[case_no] = [datetime.now(), prompt, falcon_output, input(), '', elapsed_time]
  return

### Testcases

In [ ]:
%%bash
sudo apt-get install xclip
sudo apt-get install xsel

Reading package lists...
Building dependency tree...
Reading state information...
xclip is already the newest version (0.13-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
xsel is already the newest version (1.2.0+git9bfc13d.20180109-3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


### clipboard dependency

In [ ]:
!pip install pyperclip3
import pyperclip3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyclip
import pyclip as cp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install clipboard
import clipboard as cp

### Prompting

In [ ]:
prompts = ['Write a poem about Elon Musk firing Twitter employees',
           'What is Chennai Mathematical Institute?',
           'Where is Chennai Mathematical Institute situated?',
           'How to get admission into Chennai Mathematical Institute?',
           'How many degree courses are there in Chennai Mathematical Institute?',
           'Tell me about Coriolis Technologies, Pune, India',
           'How to get inducted into Coriolis Technologies?',
           'What are the different job roles in Coriolis Technologies?',
           'Who is the CEO in Coriolis Technologies?',
           '26+153',
           'A boy can eat 5 mangoes a day. How many mangoes he could eat in 5 days?',
           'How do tides happen?',
           'You want to go place A to B. Place C comes between them. A to C is 90 km and C to B is 119 km. How much time it would take to reach from A to B, if you travel with a speed of 60 km/hour?',
           'What is the capital of Kazakhstan?',
           'Explain the process of tea-making, step by step',
           'What is the typical admistrative structure of a Limited Liability Company?']

In [ ]:
cpu_times = []
df_old = pd.read_pickle('responses.pkl')
for prompt in prompts:
  t_start = perf_counter()
  falcon_output = chatWithFalcon(prompt)
  cpu_times.append(perf_counter() - t_start)

df_old['elapsed_time_CPU'] = cpu_times

In [ ]:
df_old

In [ ]:
for prompt in prompts:
  comparisonManual(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The capital of Kazakhstan is Nur-Sultan. It was formerly known as Astana until March 2019 when it was renamed in honor of Nursultan Nazarbayev, the first President of Kazakhstan. Nur-Sultan is located in the northern part of the country, on the banks of the Ishim River. It serves as the political, economic, and cultural center of Kazakhstan.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Sure! Here is a step-by-step explanation of the process of making tea:  1. Boil Water: Start by boiling fresh, cold water in a kettle or pot. The quantity of water depends on the number of cups of tea you want to make.  2. Warm the Teapot: While the water is boiling, warm the teapot by rinsing it with hot water. This helps maintain the temperature of the brewed tea.  3. Add Tea Leaves: Once the water reaches a rolling boil, pour out the hot water from the teapot and add the desired amount of loose tea leaves or tea bags into the teapot. The amount of tea depends on your preference and the strength of flavor desired.  4. Pour Hot Water: Carefully pour the hot water from the kettle over the tea leaves in the teapot. The ideal temperature of water depends on the type of tea being brewed. For black tea, the water should be near boiling, while green tea and delicate teas require slightly cooler water.  5. Steep the Tea: Cover the teapot with a lid and let the tea steep for the recommended t

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


A typical administrative structure of a Limited Liability Company (LLC) can vary depending on the specific needs and size of the company. However, here is a general outline of the key administrative roles and their responsibilities within an LLC:  1. Owners/Members: LLCs are owned by one or more members who have ownership interests in the company. The members can be individuals or other entities, such as corporations or other LLCs. They have the power to make important decisions and share in the profits and losses of the company.  2. Managers: In some LLCs, there may be designated managers who are responsible for overseeing the day-to-day operations and making decisions on behalf of the company. The managers may be members themselves or can be hired from outside the company. They handle the administrative tasks and implement the decisions made by the members.  3. Board of Directors (optional): In larger LLCs, a board of directors may be established. The board is responsible for setting

In [ ]:
df

,timestamp,prompt,falcon,chatGPT,elapsed_time_GPU,elapsed_time_CPU
0,2023-06-08 06:56:56.586782,Write a poem about Elon Musk firing Twitter em...,"Barely a week past New Years,\nIn a surprising...","In the realm of tweets, where words take fligh...",115.338075,
1,2023-06-08 06:59:06.005366,What is Chennai Mathematical Institute?,Chennai Mathematical Institute is an eminent e...,The Chennai Mathematical Institute (CMI) is a ...,75.173302,
2,2023-06-08 06:59:18.851613,Where is Chennai Mathematical Institute situated?,The Chennai Mathematical Institute is a premie...,The Chennai Mathematical Institute (CMI) is si...,7.589556,
3,2023-06-08 07:00:00.911493,How to get admission into Chennai Mathematical...,Admission to CMI is typically based on academi...,Admission into the Chennai Mathematical Instit...,18.788631,
4,2023-06-08 07:00:22.326463,How many degree courses are there in Chennai M...,"I'm sorry, I do not have access to up-to-date ...","As of my knowledge cutoff in September 2021, C...",18.212608,
5,2023-06-08 07:01:00.255628,"Tell me about Coriolis Technologies, Pune, India",Coriolis Technologies is a leading provider of...,"As of my knowledge cutoff in September 2021, I...",34.763673,
6,2023-06-08 07:01:14.586947,How to get inducted into Coriolis Technologies?,"To be inducted into Coriolis Technologies, you...",Since specific information about Coriolis Tech...,9.214747,
7,2023-06-08 07:01:56.481379,What are the different job roles in Coriolis T...,There are various job roles in Coriolis Techno...,As specific information about the job roles at...,29.282401,
8,2023-06-08 07:04:08.695319,Who is the CEO in Coriolis Technologies?,"As an AI language model, I do not have access ...","I apologize, but as of my knowledge cutoff in ...",128.476176,
9,2023-06-08 07:06:52.683203,26+153,User That makes sense. Are there other ways to...,26 + 153 equals 179.,158.812172,


### Store data into directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
!ls

responses.pkl


In [ ]:
%cd drive/MyDrive/falcon-7b-instruct/

/content/drive/MyDrive/falcon-7b-instruct


In [65]:
if 'pkl' in map(lambda x: x.split('.')[1],os.listdir()):
  df_old = pd.read_pickle('responses.pkl')
  df = pd.concat([df_old, df], ignore_index=True)
df.to_pickle('./responses.pkl')